In [ ]:
import os

from influxdb_client import InfluxDBClient
import pandas as pd

In [ ]:
with open('../.env_vars.sh') as file:
    for line in file:
        key, value = line.strip().split('=', 1)
        os.environ[key] = value

In [ ]:
IP = "<IP_ADDRESS>"
client = InfluxDBClient(url=f"http://{IP}:8086", token=os.environ['TOKEN'], org="imdea")

In [ ]:
query = client.query_api()

In [ ]:
bucket = "tpms-office"
host = "tpms-office"

start = "2023-10-01T00:00:00Z"
stop = "2023-12-31T23:59:59Z"

query_str = f'\
from(bucket: "{bucket}")\
  |> range(start: {start}, stop: {stop})\
  |> filter(fn: (r) => r["_measurement"] == "mqtt_consumer")\
  |> filter(fn: (r) => r["host"] == "{host}")\
  |> filter(fn: (r) => r["_field"] == "temperature_C" or r["_field"] == "temperature_F" or r["_field"] == "rssi" or r["_field"] == "snr" or r["_field"] == "pressure_PSI" or r["_field"] == "pressure_kPa" or r["_field"] == "freq" or r["_field"] == "freq1" or r["_field"] == "freq2" or r["_field"] == "moving" or r["_field"] == "maybe_battery" or r["_field"] == "centrifugal_acc" or r["_field"] == "wheel" or r["_field"] == "protocol")\
'

In [ ]:
# InfluxDB query preparation
rows = query.query_csv(org='imdea', query=query_str)

# Initial cleaning
df = pd.DataFrame.from_records(rows.to_values())
df = df.drop([0,1,2]).reset_index().drop(['index'],axis=1)
df.columns = df.iloc[0]
df = df.drop([0])

aux = df.copy()

In [ ]:
# Just in case
df = aux.copy()

In [ ]:
##
print("STEP 1 - Converting Time...", end="")
# Convert '_time' to datetime and '_value' to numeric
df['_time'] = pd.to_datetime(df['_time'])
df = df.rename(columns={"_time": "time"})
print("Finished")

##
print("STEP 2 - Grouping values...", end="")
# Group the data by '_time' and 'topic' and create a list of dictionaries for each group
grouped = df.groupby(['time', 'topic'])
grouped_data = grouped.apply(lambda x: dict(zip(x['_field'], x['_value']))).reset_index()

# Rename the column containing dictionaries
grouped_data.rename(columns={0: 'measurements'}, inplace=True)
print("Finished")

##
# Convert the dictionaries to a DataFrame
print("STEP 3 - Converting measurements to dataframe...", end="")
measurements_df = pd.json_normalize(grouped_data['measurements'])
print("Finished")

##
print("STEP 4 - Grouping and getting car and id...", end="")
# Combine the '_time' and 'topic' columns with the measurements DataFrame
time_df = pd.concat([grouped_data[['time', 'topic']], measurements_df], axis=1)

# Split the 'topic' into 'car' and 'id'
split_topics = time_df['topic'].str.split('/', expand=True)
time_df['car'] = split_topics[1]
time_df['id'] = split_topics[2].str.upper()

# Drop the original 'topic' column
time_df = time_df.drop('topic', axis=1)
print("Finished!")

##
print("STEP 5 - Saving to csv...",end="")
time_df.to_csv(f'../data/{bucket}.csv')
print(f"Data saved to: ../data/{bucket}.csv")